In [1]:
import pandas as pd

In [4]:
turnstile_200905 = pd.read_csv("MTA_Data/turnstile_200905.txt")
turnstile_200912 = pd.read_csv("MTA_Data/turnstile_200912.txt")
turnstile_200919 = pd.read_csv("MTA_Data/turnstile_200919.txt")

In [8]:
turnstiles = pd.concat([turnstile_200905, turnstile_200912, turnstile_200919])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2020,00:00:00,REGULAR,7450785,2533372
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2020,04:00:00,REGULAR,7450788,2533375
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2020,08:00:00,REGULAR,7450798,2533391
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2020,12:00:00,REGULAR,7450826,2533423
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,08/29/2020,16:00:00,REGULAR,7450910,2533454
...,...,...,...,...,...,...,...,...,...,...,...
209656,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/18/2020,05:00:00,REGULAR,5554,541
209657,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/18/2020,09:00:00,REGULAR,5554,541
209658,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/18/2020,13:00:00,REGULAR,5554,541
209659,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,09/18/2020,17:00:00,REGULAR,5554,541
